In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit


In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [3]:
def huber_regression_lags_only(
    series,
    district_name,
    max_lags=3,
    differencing=True,
    epsilon=1.35,
    alpha=0.0001,
    max_iter=1000,
    use_cv=False,
    cv_params=None,
    random_state=None
):
    """
    Robust Huber regression for time series forecasting with:
    - Automatic hyperparameter tuning via grid search
    - Outlier-resistant modeling
    - Enhanced differencing checks
    """
    # Create directory structure
    os.makedirs('HuberRegression', exist_ok=True)
    
    # 1. Stationarity and Differencing (fixed adf_result check)
    original_series = series.copy()
    d = 0
    if differencing:
        adf_result = adfuller(series.dropna())
        if adf_result[1] > 0.05:  # Fixed p-value access
            d = 1
            series = series.diff().dropna()

    # 2. Feature Engineering - Lag features with type consistency
    df = pd.DataFrame({'y': series.astype(np.float64)})
    for lag in range(1, max_lags + 1):
        df[f'lag_{lag}'] = df['y'].shift(lag).astype(np.float64)
    df = df.dropna()

    # 3. Time-based split with validation
    train_size = int(len(df) * 0.8)
    train = df.iloc[:train_size]
    test = df.iloc[train_size:]

    X_train = train.drop(columns=['y'])
    y_train = train['y']
    X_test = test.drop(columns=['y'])
    y_test = test['y']

    # 4. Model training with optional CV
    if use_cv:
        # Default parameter grid
        param_grid = cv_params or {
            'epsilon': [1.0, 1.35, 1.5, 2.0],
            'alpha': [0.0001, 0.001, 0.01, 0.1]
        }
        
        grid_search = GridSearchCV(
            HuberRegressor(max_iter=max_iter),
            param_grid,
            cv=5,
            scoring='neg_mean_squared_error'
        )
        grid_search.fit(X_train, y_train)
        model = grid_search.best_estimator_
        best_epsilon = model.epsilon
        best_alpha = model.alpha
    else:
        model = HuberRegressor(
            epsilon=epsilon,
            alpha=alpha,
            max_iter=max_iter
        )
        model.fit(X_train, y_train)
        best_epsilon = epsilon
        best_alpha = alpha

    # 5. Forecasting with type consistency
    pred_test = model.predict(X_test).astype(np.float64)

    # 6. Inverse differencing with edge case handling
    if d == 1:
        last_train_value = original_series.iloc[len(original_series) - len(test) - 1]
        pred_test = np.cumsum(pred_test) + last_train_value
        y_test = original_series.iloc[-len(test):]

    # 7. Enhanced DataFrame with confidence intervals
    forecast_df = pd.DataFrame({
        'district': district_name,
        'date': y_test.index,
        'actual': y_test.values,
        'forecast': pred_test,
        'residual': y_test.values - pred_test
    })

    # 8. Robust CSV saving
    forecast_csv_path = os.path.join('HuberRegression', 'huber_forecasts.csv')
    forecast_df.to_csv(
        forecast_csv_path,
        mode='a',
        header=not os.path.exists(forecast_csv_path),
        index=False
    )
     # 9. Comprehensive metrics
    rmse = np.sqrt(mean_squared_error(forecast_df['actual'], forecast_df['forecast']))
    metrics_data = {
        'district': district_name,
        'rmse': rmse,
        'r_squared': model.score(X_train, y_train),
        'coefficients': model.coef_.tolist(),
        'intercept': model.intercept_,
        'final_epsilon': best_epsilon,
        'final_alpha': best_alpha,
        'differencing': d,
        'n_iterations': model.n_iter_
    }

    if use_cv:
        metrics_data.update({
            'best_params': grid_search.best_params_,
            'cv_results': grid_search.cv_results_
        })

    metrics_df = pd.DataFrame([metrics_data])
    
    metrics_csv_path = os.path.join('HuberRegression', 'huber_metrics.csv')
    metrics_df.to_csv(
        metrics_csv_path,
        mode='a',
        header=not os.path.exists(metrics_csv_path),
        index=False
    )

    # 10. Enhanced plotting with residuals
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
    
    # Forecast plot
    ax1.plot(original_series.index, original_series.values, label='Original Series')
    ax1.plot(forecast_df['date'], forecast_df['forecast'], 
            label='Huber Forecast', color='darkorange')
    ax1.set_title(f'Huber Regression Forecast for {district_name}\nRMSE: {rmse:.2f}')
    ax1.legend()

    # Residual plot
    ax2.bar(forecast_df['date'], forecast_df['residual'], 
           color='gray', alpha=0.7, width=5)
    ax2.axhline(0, color='red', linestyle='--')
    ax2.set_title('Forecast Residuals')
    
    plt.tight_layout()
    plt.savefig(os.path.join('HuberRegression', f'huber_forecast_{district_name}.png'))
    plt.close()

    return {
        'district': district_name,
        'forecast_df': forecast_df,
        'metrics_df': metrics_df,
        'model_params': model.get_params(),
        'feature_names': X_train.columns.tolist()
    }

In [4]:
districts = data['district'].unique()
rmse_values = []

def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I55"].asfreq('MS')
        
        results = huber_regression_lags_only(
            ts,
            district,
            use_cv=True,
            cv_params={
                'epsilon': [1.0, 1.35, 1.7],
                'alpha': [0.0001, 0.001, 0.01]
            },
            max_iter=2000
        )
        
    
        # Show results
        print("=== Metrics ===")
        print(results['metrics_df'])
        print("\n=== Forecast Data ===")
        print(results['forecast_df'].head())
    
    return results
run_for_each_district()

=== Metrics ===
     district        rmse  r_squared  \
0  AHMEDNAGAR  865.745388  -0.095359   

                                        coefficients    intercept  \
0  [0.3725247634569505, 0.07560048742639634, 0.09...  2606.418925   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.7       0.0001             0            70   

                         best_params  \
0  {'alpha': 0.0001, 'epsilon': 1.7}   

                                          cv_results  
0  {'mean_fit_time': [0.013278865814208984, 0.007...  

=== Forecast Data ===
     district       date  actual     forecast     residual
0  AHMEDNAGAR 2020-07-01  6225.0  5741.775030   483.224970
1  AHMEDNAGAR 2020-08-01  6609.0  5916.614964   692.385036
2  AHMEDNAGAR 2020-09-01  7268.0  6060.781950  1207.218050
3  AHMEDNAGAR 2020-10-01  6799.0  6377.327619   421.672381
4  AHMEDNAGAR 2020-11-01  5248.0  6287.210444 -1039.210444
=== Metrics ===
  district        rmse  r_squared  \
0    AKOLA  285.2480

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nauti\AppData\

=== Metrics ===
   district       rmse  r_squared  \
0  BULDHANA  721.96991   0.089394   

                                        coefficients  intercept  \
0  [-0.43032580834152184, -0.2383545824600653, -0... -17.083901   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.0         0.01             1            48   

                       best_params  \
0  {'alpha': 0.01, 'epsilon': 1.0}   

                                          cv_results  
0  {'mean_fit_time': [0.014159536361694336, 0.007...  

=== Forecast Data ===
   district       date  actual     forecast    residual
0  BULDHANA 2020-07-01    1172   984.576747  187.423253
1  BULDHANA 2020-08-01    1332  1002.263254  329.736746
2  BULDHANA 2020-09-01    1442   924.536535  517.463465
3  BULDHANA 2020-10-01    1664   806.934025  857.065975
4  BULDHANA 2020-11-01    1582   609.382549  972.617451
=== Metrics ===
     district        rmse  r_squared  \
0  CHANDRAPUR  277.996913   0.131104   

        

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nauti\AppData\

=== Metrics ===
  district        rmse  r_squared  \
0    JALNA  664.922936   0.037459   

                                        coefficients  intercept  \
0  [-0.23501090612445683, -0.017818663836309066, ...  23.336967   

   final_epsilon  final_alpha  differencing  n_iterations  \
0           1.35       0.0001             1            52   

                          best_params  \
0  {'alpha': 0.0001, 'epsilon': 1.35}   

                                          cv_results  
0  {'mean_fit_time': [0.011294984817504882, 0.007...  

=== Forecast Data ===
  district       date  actual     forecast    residual
0    JALNA 2020-07-01    1894  1780.126536  113.873464
1    JALNA 2020-08-01    1719  1761.282356  -42.282356
2    JALNA 2020-09-01    1730  1824.528595  -94.528595
3    JALNA 2020-10-01    2115  1843.269344  271.730656
4    JALNA 2020-11-01    1968  1784.907495  183.092505
=== Metrics ===
   district        rmse  r_squared  \
0  KOLHAPUR  1041.92673   0.037895   

            

...

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nauti\AppData\

=== Metrics ===
  district        rmse  r_squared  \
0   WARDHA  224.843152    0.05856   

                                        coefficients  intercept  \
0  [-0.12339636269013496, -0.0387267464158272, 0....   3.830457   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.0         0.01             1            36   

                       best_params  \
0  {'alpha': 0.01, 'epsilon': 1.0}   

                                          cv_results  
0  {'mean_fit_time': [0.008565139770507813, 0.004...  

=== Forecast Data ===
  district       date  actual     forecast    residual
0   WARDHA 2020-07-01    1131  1088.180147   42.819853
1   WARDHA 2020-08-01    1326  1088.912810  237.087190
2   WARDHA 2020-09-01    1431  1063.853565  367.146435
3   WARDHA 2020-10-01    1425  1048.774268  376.225732
4   WARDHA 2020-11-01    1342  1055.160362  286.839638


c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nauti\AppData\

=== Metrics ===
  district        rmse  r_squared  \
0   WASHIM  101.405981    0.06656   

                                        coefficients  intercept  \
0  [0.28788898443748073, 0.4258552219461902, 0.19...   2.903992   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.0         0.01             0            43   

                       best_params  \
0  {'alpha': 0.01, 'epsilon': 1.0}   

                                          cv_results  
0  {'mean_fit_time': [0.010014533996582031, 0.008...  

=== Forecast Data ===
  district       date  actual    forecast    residual
0   WASHIM 2020-07-01   622.0  447.631541  174.368459
1   WASHIM 2020-08-01   696.0  506.806839  189.193161
2   WASHIM 2020-09-01   678.0  582.015606   95.984394
3   WASHIM 2020-10-01   696.0  616.802735   79.197265
4   WASHIM 2020-11-01   627.0  628.871262   -1.871262
=== Metrics ===
   district        rmse  r_squared  \
0  YAVATMAL  475.542254   0.219059   

                        

{'district': 'MUMBAI SUBURBAN',
 'forecast_df':           district       date  actual     forecast     residual
 0  MUMBAI SUBURBAN 2021-02-01    3958  7196.164810 -3238.164810
 1  MUMBAI SUBURBAN 2021-03-01    4827  9619.698766 -4792.698766,
 'metrics_df':           district         rmse  r_squared  \
 0  MUMBAI SUBURBAN  4089.967775   0.727407   
 
                                         coefficients   intercept  \
 0  [-0.41627758593264325, -0.7202587935271337, -1...  830.564033   
 
    final_epsilon  final_alpha  differencing  n_iterations  \
 0           1.35        0.001             1           102   
 
                          best_params  \
 0  {'alpha': 0.001, 'epsilon': 1.35}   
 
                                           cv_results  
 0  {'mean_fit_time': [0.013823270797729492, 0.011...  ,
 'model_params': {'alpha': 0.001,
  'epsilon': 1.35,
  'fit_intercept': True,
  'max_iter': 2000,
  'tol': 1e-05,
  'warm_start': False},
 'feature_names': ['lag_1', 'lag_2', 'lag_3']}